# Introduction

In this notebook, we use Azure AutoML to train, select, and operationalize a time-series forecasting model for time-series.

Make sure you have executed the configuration notebook before running this notebook.

The dataset used is available at . More details over the dataset are give in section [Dataset](#Dataset).


# Load Packages

In [1]:
from train import *

import pandas as pd
import numpy as np
import logging
import joblib
import sys
import json

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.core.dataset import Dataset
from azureml.core.model import Model

from azureml.widgets import RunDetails

# onnx

from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants
import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

# Model deployment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)

# Check system and core SDK version number
print("System version: {}".format(sys.version))
print("SDK version:", azureml.core.VERSION)

System version: 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]
SDK version: 1.20.0


# Initialize workspace and create an Azure ML experiment

To start we need to initialize our workspace and create a Azule ML experiment. It is also to remember that accessing the Azure ML workspace requires authentication with Azure.

Make sure the config file is present at `.\config.json`. This file can be downloaded from home of Azure Machine Learning Studio.

In [2]:
#Define the workspace
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136591
aml-quickstarts-136591
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


In [3]:
#Create an experiment
experiment_name = 'automl-walmart-forecasting'
experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-walmart-forecasting,quick-starts-ws-136591,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
dic_data = {'Workspace name': ws.name,
            'Azure region': ws.location,
            'Subscription id': ws.subscription_id,
            'Resource group': ws.resource_group,
            'Experiment Name': experiment.name}

df_data = pd.DataFrame.from_dict(data = dic_data, orient='index')

df_data.rename(columns={0:''}, inplace = True)
df_data

,
Workspace name,quick-starts-ws-136591
Azure region,southcentralus
Subscription id,5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group,aml-quickstarts-136591
Experiment Name,automl-walmart-forecasting


# Create or Attach an AmlCompute cluster

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Define CPU cluster name
compute_target_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_target_name)
    print("Found existing cpu-cluster. Use it.")
except ComputeTargetException:
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",
                                                           min_nodes=1, # when innactive
                                                           max_nodes=4) # when busy
    # Create the cluster with the specified name and configuration
    compute_target = ComputeTarget.create(ws, compute_target_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# For a more detailed view of current AmlCompute status, use get_status()
print(compute_target.get_status().serialize())

Creating
Succeeded.......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-30T13:24:41.135000+00:00', 'errors': None, 'creationTime': '2021-01-30T13:22:32.959372+00:00', 'modifiedTime': '2021-01-30T13:22:48.307996+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS12_V2'}


# Dataset

## Overview

The dataset used in this project is a small subset of a much bigger dataset made available at Kaggle's competition [M5 Forecasting - Accuracy Estimate the unit sales of Walmart retail goods](https://www.kaggle.com/c/m5-forecasting-accuracy/overview/description).

The complete dataset covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. **The task is to forecast daily sales for the next 28 days.**

In order to demonstrate the use of Azure ML in forecasting we used the available data consisting of the following files and create a reduced dataset with **10 products of the 3 Texas stores of Walmart**. 

* **calendar.csv** - Contains information about the dates on which the products are sold.
* **sell_prices.csv** - Contains information about the price of the products sold per store and date.
* **sales_train_evaluation.csv** - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)

Details on how the new dataset was created can be seen in notebook [01-walmart_data_preparation](http://localhost:8888/notebooks/Capstone%20Project/notebooks/01-walmart_data_preparation.ipynb).


## Load Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

Workspace was initialized, an experiment was created as well as a compute target. Now is time to load the data into a pandas dataframe. 

The filtered data described previously was upload to GitHub. Therefore, we will use the GitHub link to upload the data. 

The time column is called `date`, so it will be parsed into the datetime type while loading the data.

In [7]:
time_column_name = 'date'
data = pd.read_csv("https://raw.githubusercontent.com/dpbac/Forecasting-Walmart-sales-with-Azure/master/data/walmart_tx_stores_10_items.csv?token=AEBB67P32BZUXMXJYVAQR63ACVPG6", parse_dates=[time_column_name])
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX,sell_price
0,HOBBIES_2_001_TX_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,nan
1,HOBBIES_2_002_TX_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,1.97
2,HOBBIES_2_003_TX_1_evaluation,HOBBIES_2_003,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,nan
3,HOBBIES_2_004_TX_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,nan
4,HOBBIES_2_005_TX_1_evaluation,HOBBIES_2_005,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,NaN,NaN,NaN,NaN,0,nan


In [8]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58230 entries, 0 to 58229
Data columns (total 15 columns):
id              58230 non-null object
item_id         58230 non-null object
dept_id         58230 non-null object
cat_id          58230 non-null object
store_id        58230 non-null object
state_id        58230 non-null object
demand          58230 non-null int64
date            58230 non-null datetime64[ns]
wm_yr_wk        58230 non-null int64
event_name_1    4740 non-null object
event_type_1    4740 non-null object
event_name_2    120 non-null object
event_type_2    120 non-null object
snap_TX         58230 non-null int64
sell_price      52938 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(10)
memory usage: 6.7+ MB


In [9]:
for store in data['store_id'].unique():
    print("Store {} has {} items".format(store,len(data['item_id'][data['store_id']=='TX_1'].unique())))

Store TX_1 has 10 items
Store TX_2 has 10 items
Store TX_3 has 10 items


Now we need to build a time-series model for the `demand` column. As we saw this reduced dataset contains many individual time-series - one for each unique combination of `store_id` and `item_id`. Note that this is also translates by the unique items in column `ìd` which consists of concatenating `ìtem_id` with `store_id`.

In order to distinguish the individual time-series, we define the `time_series_id_column_names` - the columns whose values determine the boundaries between time-series:

In [10]:
time_series_id_column_names = ['store_id', 'item_id']
nseries = data.groupby(time_series_id_column_names).ngroups
print('Data contains {0} individual time-series.'.format(nseries))

Data contains 30 individual time-series.


In [11]:
# we confirm what was said checking the unique item in column id
len(data['id'].unique())

30

## Data preparation

Before spliting data we replace NaN in events information by `"no_event"`.

In the description of the dataset given in the [Guideline](https://mofc.unic.ac.cy/wp-content/uploads/2020/03/M5-Competitors-Guide-Final-10-March-2020.docx) says: "The description of the dataset says: "If the date includes an event, the name of this event.". Therefore, I'll replace NaN values by `"no_event"`.

In [12]:
data = replace_nan_events(data)

## Split Time Series Dataset

We now split the data into a training and a testing datasets. The test set will contain the final 28 days of observed demand for each time-series. This will be used later forecast evaluation.

The function `split_train_test()` splits our time series data into training and testing following the rule that test indices need to be higher than the train indices, and higher than the test indices in the previous split.

The test split is `forecast_horizon` days long, and the testing period is `gap` number of days away from the training period. In practice, this gap is used to allow business managers to plan for the forecasted demand.

The first available day in the data (or the first day we want to start modeling from) is **2011-01-29**, and the last day is **2016-05-22**.

The image below illustrates how the data will be split by `split_train_test()`.

Consider, that our current day is **2016-04-24** and that we want to forecast 28 days without gap, i.e., we want to forecast demand from **2016-04-25** until **2016-05-22**. Therefore, in this case the parameters used will be:

* `forecast_horizon`=28 
* `gap`= 0


![](./images/time_series_split.JPG)

![](https://github.com/dpbac/Forecasting-Walmart-sales-with-Azure/blob/master/images/time_series_split.JPG)

In [13]:
df = data
forecast_horizon = 28
gap = 0

df_train, df_test = split_train_test(df,forecast_horizon, gap)

First day training dataset:2011-01-29 00:00:00
Last day training dataset:2016-04-24 00:00:00
First day test dataset:2016-04-25 00:00:00
Last day test dataset:2016-05-22 00:00:00


## Upload data to datastore

**REPLACE AND EDIT FOLLOWING TEXT**

The [Machine Learning service workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-workspace), is paired with the storage account, which contains the default data store. We will use it to upload the train and test data and create [tabular datasets](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py) for training and testing. A tabular dataset defines a series of lazily-evaluated, immutable operations to load data from the data source into tabular representation.

In [14]:
# save data locally
    
path_train = './train.csv'
path_test = './test.csv'
df_train.to_csv(path_train, index = None, header=True)
df_test.to_csv(path_test, index = None, header=True)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv', './test.csv'], 
                       target_path = 'dataset/', 
                       overwrite = True,
                       show_progress = True)

Uploading an estimated of 2 files
Uploading ./train.csv
Uploaded ./train.csv, 1 files out of an estimated total of 2
Uploading ./test.csv
Uploaded ./test.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_a90baaa707af433bbb22bbce7a29f3fa

## Create Dataset for Training

In [15]:
from azureml.core.dataset import Dataset

train_dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('dataset/train.csv'))

In [16]:
# create a pandas dataset

df = train_dataset.to_pandas_dataframe()

In [17]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX,sell_price
0,HOBBIES_2_001_TX_1_evaluation,HOBBIES_2_001,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,no_event,no_event,no_event,no_event,0,nan
1,HOBBIES_2_002_TX_1_evaluation,HOBBIES_2_002,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,no_event,no_event,no_event,no_event,0,1.97
2,HOBBIES_2_003_TX_1_evaluation,HOBBIES_2_003,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,no_event,no_event,no_event,no_event,0,nan
3,HOBBIES_2_004_TX_1_evaluation,HOBBIES_2_004,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,no_event,no_event,no_event,no_event,0,nan
4,HOBBIES_2_005_TX_1_evaluation,HOBBIES_2_005,HOBBIES_2,HOBBIES,TX_1,TX,0,2011-01-29,11101,no_event,no_event,no_event,no_event,0,nan


In [18]:
df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,demand,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_TX,sell_price
57385,HOBBIES_2_006_TX_3_evaluation,HOBBIES_2_006,HOBBIES_2,HOBBIES,TX_3,TX,0,2016-04-24,11613,no_event,no_event,no_event,no_event,0,3.97
57386,HOBBIES_2_007_TX_3_evaluation,HOBBIES_2_007,HOBBIES_2,HOBBIES,TX_3,TX,0,2016-04-24,11613,no_event,no_event,no_event,no_event,0,0.97
57387,HOBBIES_2_008_TX_3_evaluation,HOBBIES_2_008,HOBBIES_2,HOBBIES,TX_3,TX,0,2016-04-24,11613,no_event,no_event,no_event,no_event,0,3.88
57388,HOBBIES_2_009_TX_3_evaluation,HOBBIES_2_009,HOBBIES_2,HOBBIES,TX_3,TX,0,2016-04-24,11613,no_event,no_event,no_event,no_event,0,5.97
57389,HOBBIES_2_010_TX_3_evaluation,HOBBIES_2_010,HOBBIES_2,HOBBIES,TX_3,TX,0,2016-04-24,11613,no_event,no_event,no_event,no_event,0,0.97


In [19]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57390 entries, 0 to 57389
Data columns (total 15 columns):
id              57390 non-null object
item_id         57390 non-null object
dept_id         57390 non-null object
cat_id          57390 non-null object
store_id        57390 non-null object
state_id        57390 non-null object
demand          57390 non-null int64
date            57390 non-null datetime64[ns]
wm_yr_wk        57390 non-null int64
event_name_1    57390 non-null object
event_type_1    57390 non-null object
event_name_2    57390 non-null object
event_type_2    57390 non-null object
snap_TX         57390 non-null int64
sell_price      52098 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(10)
memory usage: 6.6+ MB


# AutoML Modeling

**REVIEW AND EDIT**

For forecasting tasks, AutoML uses pre-processing and estimation steps that are specific to time-series. AutoML will undertake the following pre-processing steps:
* Detect time-series sample frequency (e.g. hourly, daily, weekly) and create new records for absent time points to make the series regular. A regular time series has a well-defined frequency and has a value at every sample point in a contiguous time span 
* Impute missing values in the target (via forward-fill) and feature columns (using median column values) 
* Create features based on time series identifiers to enable fixed effects across different series
* Create time-based features to assist in learning seasonal patterns
* Encode categorical variables to numeric quantities

In this notebook, AutoML will train a single, regression-type model across **all** time-series in a given training set. This allows the model to generalize across related series. If you're looking for training multiple models for different time-series, please see the many-models notebook.

You are almost ready to start an AutoML training job. First, we need to separate the target column from the rest of the DataFrame: 

In [20]:
target_column_name = 'demand'

**IF NECESSARY**

## Customization

**REVIEW, EDIT, MINIMIZE**

The featurization customization in forecasting is an advanced feature in AutoML which allows our customers to change the default forecasting featurization behaviors and column types through `FeaturizationConfig`. The supported scenarios include:

1. Column purposes update: Override feature type for the specified column. Currently supports DateTime, Categorical and Numeric. This customization can be used in the scenario that the type of the column cannot correctly reflect its purpose. Some numerical columns, for instance, can be treated as Categorical columns which need to be converted to categorical while some can be treated as epoch timestamp which need to be converted to datetime. To tell our SDK to correctly preprocess these columns, a configuration need to be add with the columns and their desired types.
2. Transformer parameters update: Currently supports parameter change for Imputer only. User can customize imputation methods. The supported imputing methods for target column are constant and ffill (forward fill). The supported imputing methods for feature columns are mean, median, most frequent, constant and ffill (forward fill). This customization can be used for the scenario that our customers know which imputation methods fit best to the input data. For instance, some datasets use NaN to represent 0 which the correct behavior should impute all the missing value with 0. To achieve this behavior, these columns need to be configured as constant imputation with `fill_value` 0.
3. Drop columns: Columns to drop from being featurized. These usually are the columns which are leaky or the columns contain no useful data.

In [21]:
# featurization_config = FeaturizationConfig()
# featurization_config.drop_columns = ['logQuantity']  # 'logQuantity' is a leaky feature, so we remove it.
# # Force the CPWVOL5 feature to be numeric type.
# featurization_config.add_column_purpose('CPWVOL5', 'Numeric')
# # Fill missing values in the target column, Quantity, with zeros.
# featurization_config.add_transformer_params('Imputer', ['Quantity'], {"strategy": "constant", "fill_value": 0})
# # Fill missing values in the INCOME column with median value.
# featurization_config.add_transformer_params('Imputer', ['INCOME'], {"strategy": "median"})
# # Fill missing values in the Price column with forward fill (last value carried forward).
# featurization_config.add_transformer_params('Imputer', ['Price'], {"strategy": "ffill"})

In [22]:
# MAYBE - TRY WITHOUT

# featurization_config = FeaturizationConfig()
# Fill missing values in the Price column with forward fill (last value carried forward).
# featurization_config.add_transformer_params('Imputer', ['sell_price'], {"strategy": "ffill"})

## Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the ForecastingParameters class. The table below details the forecasting parameter we will be passing into our experiment.


|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**time_series_id_column_names**|The column names used to uniquely identify the time series in data that has multiple rows with the same timestamp. If the time series identifiers are not defined, the data set is assumed to be one time series.|

In [23]:
time_column_name = 'date'
forecast_horizon = 28
time_series_id_column_names = ['item_id','store_id'] # or ['id']

## AutoML Configuration - Train

TODO: Explain why you chose the automl settings and cofiguration you used below.

**REVIEW AND EDIT**

The [AutoMLConfig](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) object defines the settings and data for an AutoML training job. Here, we set necessary inputs like the task type, the number of AutoML iterations to try, the training data, and cross-validation parameters.

For forecasting tasks, there are some additional parameters that can be set in the `ForecastingParameters` class: the name of the column holding the date/time, the timeseries id column names, and the maximum forecast horizon. A time column is required for forecasting, while the time_series_id is optional. If time_series_id columns are not given, AutoML assumes that the whole dataset is a single time-series. We also pass a list of columns to drop prior to modeling. The _logQuantity_ column is completely correlated with the target quantity, so it must be removed to prevent a target leak.

The forecast horizon is given in units of the time-series frequency; for instance, the OJ series frequency is weekly, so a horizon of 20 means that a trained model will estimate sales up to 20 weeks beyond the latest date in the training data for each series. In this example, we set the forecast horizon to the number of samples per series in the test set (n_test_periods). Generally, the value of this parameter will be dictated by business needs. For example, a demand planning application that estimates the next month of sales should set the horizon according to suitable planning time-scales. Please see the [energy_demand notebook](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning/forecasting-energy-demand) for more discussion of forecast horizon.

We note here that AutoML can sweep over two types of time-series models:
* Models that are trained for each series such as ARIMA and Facebook's Prophet.
* Models trained across multiple time-series using a regression approach.

In the first case, AutoML loops over all time-series in your dataset and trains one model (e.g. AutoArima or Prophet, as the case may be) for each series. This can result in long runtimes to train these models if there are a lot of series in the data. One way to mitigate this problem is to fit models for different series in parallel if you have multiple compute cores available. To enable this behavior, set the `max_cores_per_iteration` parameter in your AutoMLConfig as shown in the example in the next cell. 


Finally, a note about the cross-validation (CV) procedure for time-series data. AutoML uses out-of-sample error estimates to select a best pipeline/model, so it is important that the CV fold splitting is done correctly. Time-series can violate the basic statistical assumptions of the canonical K-Fold CV strategy, so AutoML implements a [rolling origin validation](https://robjhyndman.com/hyndsight/tscv/) procedure to create CV folds for time-series data. To use this procedure, you just need to specify the desired number of CV folds in the AutoMLConfig object. It is also possible to bypass CV and use your own validation set by setting the *validation_data* parameter of AutoMLConfig.

Here is a summary of AutoMLConfig parameters used for training the OJ model:

|Property|Description|
|-|-|
|**task**|forecasting|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>
|**experiment_timeout_hours**|Experimentation timeout in hours.|
|**enable_early_stopping**|If early stopping is on, training will stop when the primary metric is no longer improving.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|
|**compute_target**|The remote compute for training.|
|**n_cross_validations**|Number of cross-validation folds to use for model/pipeline selection|
|**enable_voting_ensemble**|Allow AutoML to create a Voting ensemble of the best performing models|
|**enable_stack_ensemble**|Allow AutoML to create a Stack ensemble of the best performing models|
|**debug_log**|Log file path for writing debugging information|
|**featurization**| 'auto' / 'off' / FeaturizationConfig Indicator for whether featurization step should be done automatically or not, or whether customized featurization should be used. Setting this enables AutoML to perform featurization on the input to handle *missing data*, and to perform some common *feature extraction*.|
|**max_cores_per_iteration**|Maximum number of cores to utilize per iteration. A value of -1 indicates all available cores should be used

In [26]:

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id_column_names
)

automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl_walmart_forecasting_errors.log',
                             primary_metric='normalized_mean_absolute_error',
                             experiment_timeout_minutes=30,
                             training_data=train_dataset,
                             label_column_name=target_column_name,
                             compute_target=compute_target,
#                              enable_onnx_compatible_models=True,
                             enable_early_stopping=True,
#                              featurization=featurization_config, #try using and not using
                             n_cross_validations=3,
                             verbosity=logging.INFO,
                             max_cores_per_iteration=-1,
                             forecasting_parameters=forecasting_parameters)

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)
remote_run

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_8b1377ab-5ae4-4133-842d-3651879c93f7

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed, all data points are aligned with detected frequency.
              

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to cu

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
#Launch the widget to view the progress and results
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion()

## Retrieve and Save the Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#Retrieving the best model
best_run, best_model = remote_run.get_output()
print(best_model.steps)

In [ ]:
best_run_metrics = best_run.get_metrics()
print("Best run Id: ",best_run.id)
print("NMAE: ", best_run_metrics['normalized_mean_absolute_error'])
print("Other details: ")
print("Fitted model:",best_model)

In [ ]:
#Save the best model
os.makedirs('results', exist_ok=True)
automl_model_name = best_run.properties['model_name']

joblib.dump(best_model, filename="results/automl_forecasting_model.pkl")
model = remote_run.register_model(model_name=automl_model_name, description='Best AutoML Walmart forecasting model')
print("Model saved successfully!")

# Forecasting

Now that we have retrieved the best pipeline/model, it can be used to make predictions on test data. First, we remove the target values from the test set:

In [ ]:
X_test = df_test
y_test = X_test.pop(target_column_name).values

In [ ]:
X_test.head()

In [ ]:
X_test.columns

**REVIEW AND EDIT**

To produce predictions on the test set, we need to know the feature values at all dates in the test set. This requirement is somewhat reasonable for the OJ sales data since the features mainly consist of price, which is usually set in advance, and customer demographics which are approximately constant for each store over the 20 week forecast horizon in the testing data.

In [ ]:
# forecast returns the predictions and the featurized data, aligned to X_test.
# This contains the assumptions that were made in the forecast
y_predictions, X_trans = best_model.forecast(X_test)

# Evaluate

**REVIEW AND EDIT**

To evaluate the accuracy of the forecast, we'll compare against the actual sales quantities for some select metrics, included the mean absolute percentage error (MAPE). 

We'll add predictions and actuals into a single dataframe for convenience in calculating the metrics.


In [ ]:
assign_dict = {'predicted': y_predictions, target_column_name: y_test}
df_all = X_test.assign(**assign_dict)

In [ ]:
from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from matplotlib import pyplot as plt

# use automl scoring module
scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all['predicted'],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET))

print("[Test data scores]\n")
for key, value in scores.items():    
    print('{}:   {:.3f}'.format(key, value))
    
# Plot outputs
%matplotlib inline
test_pred = plt.scatter(df_all[target_column_name], df_all['predicted'], color='b')
test_test = plt.scatter(df_all[target_column_name], df_all[target_column_name], color='g')
plt.legend((test_pred, test_test), ('prediction', 'truth'), loc='upper left', fontsize=8)
plt.show()

# Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

**REVIEW AND EDIT**

_Operationalization_ (or Deployment) means getting the model into the cloud so that other can run it after you close the notebook. We will create a docker running on Azure Container Instances with the model.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print("Model name: {}".format(automl_model_name))
print(remote_run.model_id)

## Get score function

In [ ]:
# Get function which will run the forecast on serialized data from the best_run
script_file_name = 'score_forecast.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

## Deploy the model as a Web Service on Azure Container Instance (ACI)

In [ ]:
inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               auth_enabled=True, 
                                               enable_app_insights=True,
                                               tags = {'type': "automl-forecasting"},
                                               description = "Automl forecasting sample service")

aci_service_name = 'automl-walmart-forecast-01'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Gets logs from a deployed web service.
aci_service.get_logs()

In [ ]:
print("Deployment state: " + aci_service.state)
print("Scoring URI: " + aci_service.scoring_uri)
print("Authetication Key: " + aci_service.get_keys()[0])
print("Swagger URI: " + aci_service.swagger_uri)

Maybe erase the following (endpoint)

## Call Service

In [ ]:
import json
X_query = X_test.copy()

# We have to convert datetime to string, because Timestamps cannot be serialized to JSON.
X_query[time_column_name] = X_query[time_column_name].astype(str)

# The Service object accept the complex dictionary, which is internally converted to JSON string.
# The section 'data' contains the data frame in the form of dictionary.
test_sample = json.dumps({'data': X_query.to_dict(orient='records')})
response = aci_service.run(input_data = test_sample)
# translate from networkese to datascientese
try: 
    res_dict = json.loads(response)
    y_forecast_all = pd.DataFrame(res_dict['index'])
    y_forecast_all[time_column_name] = pd.to_datetime(y_forecast_all[time_column_name], unit = 'ms')
    y_forecast_all['forecast'] = res_dict['forecast']    
except:
    print(res_dict)

In [ ]:
y_forecast_all

# ONNX model

## Retrieve and save the best ONNX model

In [55]:
#Retrieve and save the best model

best_run, onnx_model = remote_run.get_output(return_onnx_model=True)
onnx_model_path = "results/best_model.onnx"
OnnxConverter.save_onnx_model(onnx_model, onnx_model_path)

OnnxConvertException: OnnxConvertException:
	Message: Requested an ONNX compatible model but the run has ONNX compatibility disabled.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Requested an ONNX compatible model but the run has ONNX compatibility disabled.",
        "target": "onnx_compatible",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        }
    }
}

## Predict with the ONNX model

In [56]:
if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    test_df = test_data.to_pandas_dataframe()
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(best_run)

    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(test_df)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Use Python version 3.6 or 3.7 to run the inference helper.')

NameError: name 'test_data' is not defined

# Delete Service

In [48]:
serv = Webservice(ws, 'automl-walmart-forecast-01')
serv

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-136591', subscription_id='5a4ab2ba-6c51-4805-8155-58759ad589d8', resource_group='aml-quickstarts-136591'), name=automl-walmart-forecast-01, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://da310598-54a4-4127-8836-462a8193e64a.southcentralus.azurecontainer.io/score, properties={'type': 'automl-forecasting'}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})

In [49]:
serv.delete()     # don't do it accidentally

# Cleaning up cluster

In [ ]:
cpu_cluster.